In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import pathlib
import os

from helper_functions import *
metadata = pd.read_csv("metadata.csv")
metadata_cfdna = metadata[metadata["Sample type"].isin(["cfDNA", "Control"])]
import warnings
warnings.filterwarnings("ignore")

topK = 500
atlas_sample_types = "Tissue,WBC"

datadir = "./datadir"
outputdir = "./outputdir_02102023"

path_to_03_output = os.path.join(outputdir, "03_output_noFDR")
path_to_04_output = os.path.join(outputdir, "04_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
path_to_15_output = os.path.join(outputdir, "15_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
os.system("mkdir -p {}".format(path_to_15_output))

atlas = pd.read_csv(os.path.join(path_to_03_output, "top{}_atlas_{}.final.csv".format(topK, atlas_sample_types)), index_col =[0])

atlas = atlas[[item for item in atlas.columns if "_y" not in item ]]
atlas.columns = [item.replace("_x", "") for item in atlas.columns]

avg_atlas = atlas.set_index("sample").fillna(0).groupby("label").mean()
avg_atlas = avg_atlas.loc[atlas_labels[atlas_sample_types]]
atlas_regions = [item for item in atlas.columns if item not in ["sample", "label"]]

In [16]:
df = pd.DataFrame()
for region in tqdm(atlas_regions):
    region_chrom = region.split("_")[0]
    region_start = int(region.split("_")[1])
    region_end = int(region.split("_")[2])
    refseq_at_cluster = get_refseq(path_to_all_fa = path_to_all_fa, 
                                        chrom = region_chrom, 
                                        start = region_start, 
                                        end = region_end + 1)
    all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_cluster)]
    cpg_coords = [item + region_start for item in all_cpg_in_cluster]
    tmpdf = pd.DataFrame(data = cpg_coords, columns = ["pos"])
    tmpdf["chrom"] = region_chrom
    tmpdf["region"] = region
    df = pd.concat([df, tmpdf], axis = 0)
    

100%|██████████████████████████████████████| 2945/2945 [00:08<00:00, 358.48it/s]


In [17]:
df.to_csv("atlas_cpgs.csv")